In [ ]:
# #On va importer les packages 
# !pip install wordcloud 
# !pip install nltk
# !pip install gensim

In [1]:
import pandas as pd
import re
import os

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess

import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Récupérations données 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
fichier = os.path.join(base_dir, 'data','intermediate', 'base_merged.csv')

df = pd.read_csv(fichier, sep=';', quotechar='"')

df.sample(5)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
252,30,300011,"\r\nLe 14 juillet dernier, à 20h22, Trystan, p...",Article d'opinion,NaN,2018-07-17 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/On-...
346,13,130054,La Suédoise FRIDA STRANNE défend son plus réce...,blog,TRADFEM,16/07/2023,La Suédoise FRIDA STRANNE défend son plus réce...,https://tradfem.wordpress.com/2023/07/16/la-su...
345,13,130053,L’Alliance LGB ne sera jamais réduite au silen...,blog,TRADFEM,16/07/2023,L’Alliance LGB ne sera jamais réduite au silence,https://tradfem.wordpress.com/2023/07/16/lalli...
326,13,130034,CANADA : Quand le transactivisme devient une p...,blog,TRADFEM,26/08/2023,CANADA : Quand le transactivisme devient une p...,https://tradfem.wordpress.com/2023/08/26/canad...
231,20,200093,De nombreux enfants souffrent du sentiment qu’...,Guide,NaN,NaN,NaN,https://cryforrecognition.be/fr/guide-concis-p...


In [9]:
# Travail des stop words
mots_fr = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent' 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–', "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où", "ni", "mais", "et", "donc", "parce", "que", "quand", "comment", "qui", "quoi",
    "à", "de", "pour", "sans", "sur"
]
# Concaténer tous les textes de la colonne 'contenu' en une seule chaîne
all_text = ' '.join(df['contenu'])

# Fonction pour enlever les mots de la liste mots_fr
def remove_words(text):
    words = word_tokenize(text.lower())  # normaliser la casse ici
    filtered_words = [word for word in words if word not in mots_fr]
    return ' '.join(filtered_words)
    
# Appliquer la fonction remove_words
df['contenu'] = df['contenu'].apply(lambda x: remove_words(x))

# Fonction pour normaliser les mots au singulier 
def singularize_word(word):
    all_words = set(all_text.split())
    # Retire le 's' à la fin du mot si présent et si la version sans 's' existe déjà strictement
    if word.lower().endswith('s') and word[:-1].lower() in all_words:
        return word[:-1]
    #Pluriels en X
    elif word.lower().endswith('x') and word[:-1].lower() in all_words:
        return word[:-1]
    #mots en al
    elif word.lower().endswith('x') and (word[:-2].lower() + 'l') in all_words:
        return word[:-2].lower() + 'l'
    #mots en ail 
    elif word.lower().endswith('x') and (word[:-2].lower() + 'il') in all_words:
        return word[:-2].lower() + 'il'
    else:
        return word
        
df['contenu'] = df['contenu'].apply(lambda x: ' '.join([singularize_word(word) for word in x.split()]))

all_text2 = ' '.join(df['contenu'])

# Fonction pour obtenir les mots les plus représentés
def get_most_common_words(text):
    words = word_tokenize(text.lower())
    freq_dist = FreqDist(words)
    return freq_dist.most_common()

# Appliquer la fonction à l'ensemble du texte
most_common_words = get_most_common_words(all_text2)
most_common_words

KeyboardInterrupt: 

In [ ]:
mots_fr2 = ["m.", '—', '``', "autre", "peut", "peuvent", "ont", "a", "suis", "es", "sont", "sommes", "êtes", "me", "te", "se", "tant", "alors", "toute", "tout", "tous", "toutes", "façon", "chose", "aujourd", "hui",
           "façon", "soit", "lui", "moi"]

def remove_words2(text):
    words = word_tokenize(text.lower())  # normaliser la casse ici
    filtered_words = [word for word in words if word not in mots_fr2]
    return ' '.join(filtered_words)
    
# Appliquer la fonction remove_words
df['contenu'] = df['contenu'].apply(lambda x: remove_words2(x))

all_text2 = ' '.join(df['contenu'])

most_common_words = get_most_common_words(all_text2)
most_common_words[:50]

In [70]:
#Pour corriger le genre 
def masc_word(word):
    # Retire le 'e' à la fin du mot si présent et si la version sans 'e' existe déjà strictement
    if word.lower().endswith('e') and word[:-1].lower() in set(all_text.split()):
        return word[:-1]
    # Adaptation au cas des mots en -er (ex : berger --> bergère)
    elif word.lower().endswith('re') and word[-3].lower() in ['e', 'è', 'é'] and (word[:-3].lower() + 'er') in set(all_text.split()):
        return word[:-3] + 'er'
    #Cas de mots en f et p (loup --> louve, veuf --> veuve) 
    elif (word.lower().endswith('ve') and ((word[-2].lower() + 'f') in set(all_text.split()) or (word[-2].lower() + 'p') in set(all_text.split()))):
        if word[-2] + 'f' in set(all_text.split()): 
            return word[-2] + 'f'
        if word[-2] + 'p' in set(all_text.split()): 
            return word[-2] + 'p'
    #Cas de la double consonne 
    elif word.lower().endswith('e') and word[-2] == word[-3] and word[:-2].lower() in set(all_text.split()):
        return word[:-2]
    #chameau --> Chamelle 
    elif word.lower().endswith('elle') and (word[:-4].lower() + 'eau') in set(all_text.split()):
        return word[:-4] + 'eau'
    #époux --> Epouse 
    elif word.lower().endswith('se') and (word[:-2].lower() + 'x') in set(all_text.split()):
        return word[:-2] + 'x' 
    else:
       
df['contenu'] = df['contenu'].apply(lambda x: ' '.join([singularize_word(word) for word in x.split()]))

In [7]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_clean.csv'), index=False)

In [8]:
#Récupérations données 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
fichier = os.path.join(base_dir, 'data', 'base_clean.csv')

df = pd.read_csv(fichier)

In [17]:
# Tokenisation et suppression des stop words
stop_words = set(stopwords.words('french'))
df['tokenized_content'] = df['contenu'].apply(lambda x: [word for word in simple_preprocess(x) if word not in stop_words])

# Création d'un dictionnaire et d'un corpus
dictionary = corpora.Dictionary(df['tokenized_content'])
corpus = [dictionary.doc2bow(doc) for doc in df['tokenized_content']]

# Entraînement du modèle LDA
lda_model = models.LdaModel(corpus, num_topics=25, id2word=dictionary, passes=15)

In [18]:
# Fonction d'affichage
def print_topics(model, n_top_words):
    topics = model.print_topics(num_words=n_top_words)
    for topic_idx, topic in topics:
        print(f"\nTopic #{topic_idx}: {topic}")

# Affichage des résultats
print_topics(lda_model, 15)


Topic #15: 0.032*"femelle" + 0.022*"mâle" + 0.014*"humanité" + 0.012*"corps" + 0.012*"rapport" + 0.010*"vie" + 0.010*"violence" + 0.009*"pouvoir" + 0.009*"humaine" + 0.009*"sexuation" + 0.009*"puissance" + 0.009*"réel" + 0.008*"décolonisation" + 0.008*"monde" + 0.008*"femme"

Topic #3: 0.007*"alliance" + 0.006*"mermaids" + 0.005*"lgb" + 0.004*"transactiviste" + 0.004*"travesti" + 0.004*"pancarte" + 0.004*"organisation" + 0.003*"passé" + 0.003*"clinique" + 0.003*"promotion" + 0.003*"commission" + 0.003*"caritative" + 0.003*"ajouté" + 0.003*"juridique" + 0.003*"police"

Topic #12: 0.012*"violence" + 0.010*"femme" + 0.010*"hanmer" + 0.005*"féministe" + 0.005*"homme" + 0.004*"royaume" + 0.004*"domestique" + 0.004*"uni" + 0.004*"universitaire" + 0.003*"nationale" + 0.003*"association" + 0.003*"tre" + 0.003*"women" + 0.003*"année" + 0.003*"enfant"

Topic #22: 0.011*"femme" + 0.010*"chromosome" + 0.009*"sexe" + 0.008*"sexuel" + 0.007*"genre" + 0.006*"chez" + 0.006*"homme" + 0.005*"victime" +